## Random Forest Base

### Data Preparation

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
#import hvplot.xarray
import sys

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

sys.path.insert(0, '../../src')

from utils import df_to_xarray,read_xarray

In [4]:
# Reading Data
dir_name="../../data/data1"
chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)



In [5]:
# Creating one singular df
data_read=xr.merge([mld.MLD,mld.MLD_socat,sst.SST,sst.SST_socat,sss.SSS,sss.SSS_socat,xco2])

In [6]:
tmp_data=data_read.to_dataframe().reset_index()


In [7]:
tmp_data=tmp_data.drop(columns=['bnds','TLONG', 'TLAT', 'time_bnds'])

In [8]:
chl_data=chl.Chl.to_dataframe().reset_index()
chl_data_socat=chl.Chl_socat.to_dataframe().reset_index()
pco2_data=pco2.pCO2.to_dataframe().reset_index()
pco2_data_socat=pco2.pCO2_socat.to_dataframe().reset_index()

In [9]:
tmp_data["Chl_socat"]=chl_data_socat["Chl_socat"]
tmp_data["Chl"]=chl_data["Chl"]
tmp_data["pCO2_socat"]=pco2_data_socat["pCO2_socat"]
tmp_data["pCO2"]=pco2_data["pCO2"]

In [10]:
features_socat = ['time','xlon', 'ylat','MLD_socat', 'SST_socat', 'SSS_socat','Chl_socat', 'XCO2','pCO2_socat']
features = ['time','xlon', 'ylat','MLD','SST','SSS','Chl','XCO2','pCO2']

# create separate dataframe for socat
combined_socat=tmp_data[features_socat]
combined=tmp_data[features]

In [11]:
# drop rows where pco2 or pco2_socat == NA or 0
combined_socat.dropna(subset = ["pCO2_socat"], inplace=True)
combined_socat= combined_socat[combined_socat['pCO2_socat']!=0]

combined.dropna(subset = ["pCO2"], inplace=True)
combined= combined[combined['pCO2']!=0]



/burg/glab/users/sk4973/venv/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [12]:
combined_socat= combined_socat[combined_socat['MLD_socat']!=0]
combined_socat=combined_socat.dropna()

In [13]:
#separating X and y
X_socat=combined_socat.iloc[:,3:-1]
X=combined.iloc[:,3:-1]
y=combined.loc[:,'pCO2']
y_socat=combined_socat.loc[:,'pCO2_socat']


### Imputation
We can save 6452246 rows through imputation.

Try Building a Custom Imputation based on lon and lat?
https://towardsdatascience.com/coding-a-custom-imputer-in-scikit-learn-31bd68e541de


Also, consider using Hurdle Model?

https://geoffruddock.com/building-a-hurdle-regression-estimator-in-scikit-learn/

#### Two Different Imputation Methods
- KNNImputer: fill in the average of the 2 nearest neighbors, takes a long time to train
- Simple Imputer: just fill in using the average

In [17]:
X_socat.describe()
#get rid of 0s by converting it to NANs


,MLD_socat,SST_socat,SSS_socat,Chl_socat,XCO2
count,2621.000000,2621.000000,2621.000000,2621.000000,2621.000000
mean,51.917508,15.487437,33.457011,0.225942,387.387939
std,50.263286,9.490092,1.526085,0.515163,12.510401
min,7.868806,-1.825340,26.063950,0.001966,343.963409
25%,23.666689,8.050923,32.406136,0.104000,378.810028
50%,39.688725,15.920362,33.653847,0.135566,389.602814
75%,69.267616,24.034296,34.578911,0.190518,397.187469
max,1193.783569,30.828823,36.894070,6.348923,406.971283


In [18]:
X.describe()

,MLD,SST,SSS,Chl,XCO2
count,1.083822e+07,1.083822e+07,1.083822e+07,1.729047e+07,1.729047e+07
mean,6.533868e+01,1.319081e+01,3.375592e+01,3.449500e-01,3.701860e+02
std,5.373071e+01,1.153312e+01,1.569614e+00,8.521562e-01,1.869547e+01
min,7.500032e+00,-1.936021e+00,1.376396e+01,-4.092084e-01,3.408485e+02
25%,3.466645e+01,6.017284e-01,3.324879e+01,1.087646e-01,3.547707e+02
50%,5.551451e+01,1.352542e+01,3.385484e+01,1.580932e-01,3.681608e+02
75%,8.369460e+01,2.482988e+01,3.459950e+01,2.111136e-01,3.854302e+02
max,1.868325e+03,3.407636e+01,4.305632e+01,1.467028e+01,4.072084e+02


In [174]:
#We can save this many rows through imputation.
# These rows have xCO2, pXO2 and CHL, but no MLD, SSS, SST
combined_socat.isna().sum()

# get rid of NAs

time               0
xlon               0
ylat               0
MLD_socat     112674
SST_socat     112674
SSS_socat     112674
Chl_socat          0
XCO2               0
pCO2_socat         0
dtype: int64

In [18]:
# Two Different Imputation Methods

# KNNImputer
# from sklearn.impute import KNNImputer
# imp = KNNImputer(n_neighbors=2)
# X=imp.fit_transform(X)
# X_socat=imp.fit_transform(X_socat)

# SimpleImputer
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X=imp.fit_transform(X)
X_socat=imp.fit_transform(X_socat)

### Modeling - Random Forest: full




In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 73)

regressor=RandomForestRegressor(n_estimators=20, random_state=42, verbose=3,n_jobs=-1, 
                                max_depth=10,warm_start= True)
regressor.fit(X_train, y_train)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [24]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
forest_scores = cross_val_score(regressor, X_train, y_train,
                                scoring="neg_mean_squared_error", cv=3)
forest_rmse_scores = np.sqrt(-forest_scores)

display_scores(forest_rmse_scores)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Paralle

Scores: [38.16895403 38.16318419 38.23905966]
Mean: 38.190399292606735
Standard deviation: 0.0344886065303208


[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.0s finished


### Final Result: full

Socat RMSE: 

Whole Grid Rmse: 

In [25]:
# On Socat
y_pred=regressor.predict(X_test)
test_mse=mean_squared_error(y_test, y_pred,squared=True)
np.sqrt(test_mse)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.3s finished


38.23882857143651

In [27]:
## The whole grid
y_pred=regressor.predict(X)
final_test_rmse=np.sqrt(mean_squared_error(y, y_pred,squared=True))
error=y-y_pred

final_test_rmse

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    2.3s finished


38.18342107365459

building tree 2 of 20
building tree 6 of 20
building tree 10 of 20
building tree 15 of 20
building tree 19 of 20
building tree 3 of 20
building tree 8 of 20
building tree 12 of 20
building tree 16 of 20
building tree 20 of 20
building tree 4 of 20
building tree 7 of 20
building tree 10 of 20
building tree 15 of 20
building tree 19 of 20
building tree 4 of 20
building tree 5 of 20
building tree 9 of 20
building tree 13 of 20
building tree 17 of 20
building tree 1 of 20
building tree 5 of 20
building tree 10 of 20
building tree 14 of 20
building tree 18 of 20
building tree 2 of 20
building tree 6 of 20
building tree 11 of 20
building tree 14 of 20
building tree 18 of 20
building tree 3 of 20
building tree 8 of 20
building tree 12 of 20
building tree 16 of 20
building tree 20 of 20
building tree 4 of 20
building tree 7 of 20
building tree 11 of 20
building tree 15 of 20
building tree 19 of 20
building tree 3 of 20
building tree 8 of 20
building tree 12 of 20
building tree 16 of 20
buildin

### Modeling - Random Forest: Socat


Uses train_test_split build into sklearn.model_selection


By default this method shuffles the data (30% = testing 70%=training/validation)
- Will test validation via 7-fold cross validation

Train  = 70%, Test   = 30%

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_socat, y_socat, test_size=0.3, random_state= 73)


In [ ]:
regressor=RandomForestRegressor(n_estimators=20, random_state=42)
regressor.fit(X_train, y_train)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

forest_scores = cross_val_score(regressor, X_train, y_train,
                                scoring="neg_mean_squared_error", cv=7)
forest_rmse_scores = np.sqrt(-forest_scores)

In [23]:
display_scores(forest_rmse_scores)

Scores: [20.48462476 22.73208136 24.10310485 23.69893613 20.18031386 22.14916205
 24.98671208]
Mean: 22.619276440571895
Standard deviation: 1.6780589138771642


In [24]:
#Fine Tuning Using RandomizedSearch

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=20, high=50),
        'max_features': randint(low=1, high=6),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=7, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=7, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1391a1ca0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1391a1ee0>},
                   random_state=42, scoring='neg_mean_squared_error')

In [25]:
final_model =rnd_search.best_estimator_


### Final Result

Test Set RMSE: 23.818525580843453

Whole Grid Rmse: 40.38668015093895

In [26]:
y_pred=final_model.predict(X_test)
test_mse=mean_squared_error(y_test, y_pred,squared=True)
np.sqrt(test_mse)

23.818525580843453

In [27]:
## The whole grid
y_pred=final_model.predict(X)
final_test_rmse=np.sqrt(mean_squared_error(y, y_pred,squared=True))
error=y-y_pred

In [28]:
final_test_rmse

40.38668015093895

### Visualization of the Residual

In [29]:
combined["residual"]=np.abs(error)

In [30]:
result_data=combined[["time","xlon","ylat","residual"]]

In [31]:
cols=result_data.columns.tolist()
cols=[cols[0],cols[2],cols[1],cols[3]]
result_data=result_data[cols]
result_data.columns=['time','lat','lon','residual']

In [32]:
result_data['time'].iloc[0]

cftime.DatetimeNoLeap(1982, 2, 1, 0, 0, 0, 0, has_year_zero=True)

In [36]:
result_data

,time,lat,lon,residual
4499,1982-02-01 00:00:00,89.5,24.5,187.653834
4500,1982-02-01 00:00:00,-89.5,25.5,176.520662
4501,1982-02-01 00:00:00,-88.5,25.5,178.350852
4502,1982-02-01 00:00:00,-87.5,25.5,180.891012
4503,1982-02-01 00:00:00,-86.5,25.5,183.319747
...,...,...,...,...
27280795,2017-02-01 00:00:00,85.5,359.5,22.196348
27280796,2017-02-01 00:00:00,86.5,359.5,20.499094
27280797,2017-02-01 00:00:00,87.5,359.5,18.863136
27280798,2017-02-01 00:00:00,88.5,359.5,17.434593


In [40]:

def tmp_df(df_in=None):
    dates = xr.cftime_range(start=f'1982-02-01', end=f'2018-12-01',freq='MS') 
    ds_skeleton = xr.Dataset({'lon':np.arange(0.5, 360, 1), 
                              'lat':np.arange(-89.5, 90, 1),
                              'time':dates})    
    # make dataframe
    skeleton = df_in.reset_index()[['lon','lat','time']]
    # Merge predictions with df_all dataframe
    df_out = skeleton.merge(df_in, how = 'left', on = ['lon','lat','time'])
    # convert to xarray dataset
    # old way to `dimt, = ds_skeleton.time.shape` ect. to get dimensions
    # then reshape  `df_out.values.reshape(dim_lat, dim_lon, dim_time)`
    # finally create a custom dataset
    df_out.set_index(['lon','lat','time'], inplace=True)
    ds = df_out.to_xarray()
    return ds

ds=tmp_df(result_data)
              


In [41]:
a=ds.residual.hvplot(groupby="time",width=512,height=512, widget_type='scrubber', widget_location='bottom')
a

Column
    [0] HoloViews(DynamicMap, widget_location='bottom', widget_type='scrubber')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Player(end=420, width=550)
        [2] HSpacer()